- ETL - EDA 

Librerías Necesarias

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Los archivos se encuentran en este link: https://drive.google.com/drive/folders/1nJ9ZMj6E6zh6McC9NwCA6KopfUIOG_1O 
Para su ejecución debe descargarlos y pegarlos en una carpeta llamada ' Datasets'

Leemos los archivos provistos: 

In [4]:
pathtrain = 'Datasets/train.parquet'
pathtest = 'Datasets/test.parquet'

In [5]:
dftrain = pd.read_parquet(pathtrain)
dftest = pd.read_parquet(pathtest)

In [6]:
dftrain

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346474,7050982281,https://evansville.craigslist.org/apa/d/evansv...,evansville,https://evansville.craigslist.org,672,apartment,660,1,1.0,1,...,0,0,0,laundry on site,None,https://images.craigslist.org/00x0x_9oaHKZ8Ilp...,"The pool, the office, the 24-hour on-site laun...",37.9591,-87.5293,in
346475,7049418251,https://sandiego.craigslist.org/ssd/apa/d/chul...,san diego,https://sandiego.craigslist.org,2122,apartment,1099,2,2.0,1,...,0,0,0,w/d in unit,off-street parking,https://images.craigslist.org/00O0O_6QsU4p5WMh...,Seize your chance to live in our beautiful apa...,32.6279,-117.0370,ca
346476,7048268235,https://columbia.craigslist.org/apa/d/columbia...,columbia,https://columbia.craigslist.org,1014,apartment,1104,2,2.0,1,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00000_dpNWqzYhGK...,Tucked into the rolling landscape of Southeast...,33.9659,-80.9355,sc
346477,7026721229,https://nd.craigslist.org/apa/d/minot-open-con...,north dakota,https://nd.craigslist.org,935,apartment,1050,2,2.0,0,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00Z0Z_65g6Cty1RX...,Enjoy living at Southwood Apartments! Located ...,48.1995,-101.2800,nd


EXTRACCIÓN DE COLUMNAS IMPORTANTES
- Columna 'price' como etiqueta
- NO se tomarán en cuenta Columnas con respecto a una indexacion, código o ordenanza interna al estudio
- NO se extrae la información a aquellos valores que no sean contables/medibles
- Se excluirán columnas con poca correlación y que no poseen independencia lineal con la columna price

Columnas Principales 
- sqfeet: Metros cuadrados de la propiedad.
- beds: Cantidad de dormitorios.
- baths: Cantidad de baños.
- comes_furnished: Si la propiedad viene amueblada toma el valor 1, 0 para caso contrario.
- price: Precio de la propiedad en petrodólares.

In [7]:
dftrain1 = dftrain[['sqfeet','beds','baths','comes_furnished','price']].apply(lambda x: x.astype(int, errors='ignore').where(x.notnull(), 0))

Designamos una nueva variable en la cual sea el mismo dataframe y solo estén las columnas principales(sean enteros) 

- Analisis de la Columna 'sqfeet' 

In [8]:
print('Datos únicos = ' + str(dftrain1['sqfeet'].unique()))
print('Cantidad de elemetos únicos = ' + str(len(dftrain1['sqfeet'].unique())))
print('Media = ' + str(dftrain1['sqfeet'].mean()))
print('Varianza = ' + str(dftrain1['sqfeet'].var()))
print('Desviación estandar = ' + str(dftrain1['sqfeet'].std()))
print('Valor máximo = ' + str(dftrain1['sqfeet'].max()))
print('tipo = ' + str(dftrain.loc[dftrain['sqfeet'] == dftrain['sqfeet'].max(), 'type']))
print('Valor mínimo = ' + str(dftrain1['sqfeet'].min()))


Datos únicos = [1200  694  900 ... 5416 3423 3516]
Cantidad de elemetos únicos = 3193
Media = 1066.3259100840166
Varianza = 407449297.4053155
Desviación estandar = 20185.373353131607
Valor máximo = 8388607
tipo = 249237    apartment
Name: type, dtype: object
Valor mínimo = 0


Al ver los paramétros podemos notar que su varianza es mayor que su media lo cual puede significar la presencia de outliers, pero al ver su máximo es de 95mil metros cuadrados lo cual puede ser posible una propiedad de esas dimensiones pero al ver el tipo de la vivienda nos damos cuenta que es un apartamento lo cual es imposible.

In [9]:
while dftrain1['sqfeet'].max() > 10*(dftrain1['sqfeet'].mean() + dftrain1['sqfeet'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    dftrain1 = dftrain1.replace(dftrain1['sqfeet'].max() , int(dftrain1['sqfeet'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción

print('Valor máximo = ' + str(dftrain1['sqfeet'].max()))
print('Media = ' + str(dftrain1['sqfeet'].mean()))
print('Desviación estandar = ' + str(dftrain1['sqfeet'].std()))

Valor máximo = 14000
Media = 994.9758080576312
Desviación estandar = 414.1198705908561


Podemos notar mediante esta transformación los cambios favorables en los parametros

- Analisis de la Columna 'beds' 

In [10]:
print('Datos únicos = ' + str(dftrain1['beds'].unique()))
print('Cantidad de elemetos únicos = ' + str(dftrain1['beds'].unique()))
print('Media = ' + str(dftrain1['beds'].mean()))
print('Varianza = ' + str(dftrain1['beds'].var()))
print('Desviación estandar = ' + str(dftrain1['beds'].std()))
print('Valor máximo = ' + str(dftrain1['beds'].max()))
print('tipo = ' + str(dftrain.loc[dftrain['beds'] == dftrain['beds'].max(), 'type']))
print('Valor mínimo = ' + str(dftrain1['beds'].min()))

Datos únicos = [   2    1    3    5    4    0    6    8    7 1000 1100]
Cantidad de elemetos únicos = [   2    1    3    5    4    0    6    8    7 1000 1100]
Media = 1.9031889378577056
Varianza = 10.002573515613
Desviación estandar = 3.1626845425386643
Valor máximo = 1100
tipo = 316209    apartment
Name: type, dtype: object
Valor mínimo = 0


Al ver los parámetros podemos notar a simple vista de la presencia de un outlier en el cual presume que hay 1100 camas en un apartamento.

In [11]:
while dftrain1['beds'].max() > 10*(dftrain1['beds'].mean() + dftrain1['beds'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    dftrain1 = dftrain1.replace(dftrain1['beds'].max() , int(dftrain1['beds'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción

print('Valor máximo = ' + str(dftrain1['beds'].max()))
print('Media = ' + str(dftrain1['beds'].mean()))
print('Desviación estandar = ' + str(dftrain1['beds'].std()))

Valor máximo = 8
Media = 1.8942504451929265
Desviación estandar = 0.8785602767695696


Con el codigo usado eliminamos el outlier antes mencionados, es decir transforma los valores que sobrepasan 10 veces la suma de la media y su desviación.

- Analisis de la columna 'baths'

In [12]:
print('Datos únicos = ' + str(dftrain1['baths'].unique()))
print('Cantidad de elemetos únicos = ' + str(dftrain1['baths'].unique()))
print('Media = ' + str(dftrain1['baths'].mean()))
print('Varianza = ' + str(dftrain1['baths'].var()))
print('Desviación estandar = ' + str(dftrain1['baths'].std()))
print('Valor máximo = ' + str(dftrain1['baths'].max()))
print('tipo = ' + str(dftrain.loc[dftrain['baths'] == dftrain['baths'].max(), 'type']))
print('Valor mínimo = ' + str(dftrain1['baths'].min()))

Datos únicos = [ 2  1  0  3  4  5  6  7 25 35  8 75]
Cantidad de elemetos únicos = [ 2  1  0  3  4  5  6  7 25 35  8 75]
Media = 1.4262076489484212
Varianza = 0.35018980733116895
Desviación estandar = 0.5917683730406424
Valor máximo = 75
tipo = 316209    apartment
Name: type, dtype: object
Valor mínimo = 0


Podemos notar la presencia a simple vista de un outlier lo cual presume 75 baños en un departamento.

In [13]:
while dftrain1['baths'].max() > 10*(dftrain1['baths'].mean() + dftrain1['baths'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    dftrain1 = dftrain1.replace(dftrain1['baths'].max() , int(dftrain1['baths'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción

print('Valor máximo = ' + str(dftrain1['baths'].max()))
print('Media = ' + str(dftrain1['baths'].mean()))
print('Desviación estandar = ' + str(dftrain1['baths'].std()))

Valor máximo = 8
Media = 1.4258266734780463
Desviación estandar = 0.5742044781931906


Con el mismo código se solucionó el tema del outlier.(En casos de valores únicos discretos y contables podríamos eliminar/transformar buscando esa linea, pero no automatizariamos el proceso ante un futuro cambio en el dataframe) En este caso el mismo criterio fue óptimo para esta tarea.

- Analisis de la columna 'comes_furnished'

In [14]:
dftrain1['comes_furnished'].unique()

array([0, 1])

Como podemos notar los valores son binarios la cual no es necesario transformarlos.

- Analisis de la columna 'price'

In [15]:
print('Datos únicos = ' + str(dftrain1['price'].unique()))
print('Cantidad de elemetos únicos = ' + str(dftrain1['price'].unique()))
print('Media = ' + str(dftrain1['price'].mean()))
print('Varianza = ' + str(dftrain1['price'].var()))
print('Desviación estandar = ' + str(dftrain1['price'].std()))
print('Valor máximo = ' + str(dftrain1['price'].max()))
print('tipo = ' + str(dftrain.loc[dftrain['price'] == dftrain['price'].max(), 'type']))
print('Valor mínimo = ' + str(dftrain1['price'].min()))

Datos únicos = [1350 1115 1129 ... 4756 3221 5015]
Cantidad de elemetos únicos = [1350 1115 1129 ... 4756 3221 5015]
Media = 9647.14643311716
Varianza = 22123600007905.51
Desviación estandar = 4703573.110721839
Valor máximo = 2768307249
tipo = 106214    apartment
Name: type, dtype: object
Valor mínimo = 0


Al estar todo en una misma moneda(Dólares) no hay que hacer cambio alguno sobre su modulo. También por otro lado se registra un outlier de 2768307249 de dólares por un departamento. Lo cual volveremos a usar el mismo criterio.

In [16]:
while dftrain1['price'].max() > 10*(dftrain1['price'].mean() + dftrain1['price'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    dftrain1 = dftrain1.replace(dftrain1['price'].max() , int(dftrain1['price'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción

print('Valor máximo = ' + str(dftrain1['price'].max()))
print('Media = ' + str(dftrain1['price'].mean()))
print('Desviación estandar = ' + str(dftrain1['price'].std()))

Valor máximo = 18010
Media = 1166.250930070798
Desviación estandar = 643.1641337014569


CREAMOS NUEVAS COLUMNAS
- category_price: En la cual tiene que cumplir:
    - 'low': Para precios entre 0 y 999 dólares.
    - 'medium': Para precios entre 1000 y 1999 dólares.
    - 'high': Para precios desde 2000 dólares en adelante.
- target = '1' para valores 'low'  y '0' para los demás.

In [17]:
dftrain1['category_price'] = dftrain1.apply(lambda row: 'low' if row["price"] > 2000 else ('medium' if (row["price"] > 1000 and row["price"] < 1999) else 'low'), axis=1)

In [18]:
dftrain1['target'] = dftrain1.apply(lambda row: 1 if row["category_price"] == 'low' else 0, axis=1)

- PREDICCIÓN SUPERVISADA

In [22]:
X = dftrain1[['sqfeet','beds','baths','comes_furnished']]
y = dftrain1['target']

In [20]:
#Importamos el separador
from sklearn.model_selection import train_test_split

In [23]:
#Separamos el %30 de los datos para el test y lo restante lo entrenamos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [24]:
from sklearn.tree import DecisionTreeClassifier

# Instanciamos un objeto de la clase DecisionTreeClassifier

Arbol = DecisionTreeClassifier(max_depth = 4, random_state = 0)

In [25]:
# Entrenamos el modelo

Arbol.fit(X_train.values,y_train.values)

DecisionTreeClassifier(max_depth=4, random_state=0)

In [26]:
# Predecimos los atributos apartados para el testeo

y_pred = Arbol.predict(X_test.values)

In [27]:
from sklearn.metrics import accuracy_score , recall_score

Accuracy = accuracy_score(y_pred,y_test)
Recall = recall_score(y_pred,y_test)

print(Accuracy , Recall)

0.6305895482182714 0.6438242156731613


In [28]:
#Aplicamos las mismas transformaciones al Dataframe de Test 
datatest = dftest[['sqfeet','beds','baths','comes_furnished']].apply(lambda x: x.astype(int, errors='ignore').where(x.notnull(), 0))

In [29]:
#Columna beds
while datatest['beds'].max() > 10*(datatest['beds'].mean() + datatest['beds'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    datatest = datatest.replace(datatest['beds'].max() , int(datatest['beds'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción


In [30]:
#Columna baths
while datatest['baths'].max() > 10*(datatest['baths'].mean() + datatest['baths'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    datatest = datatest.replace(datatest['baths'].max() , int(datatest['baths'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción


In [31]:
#Columna sqfeet
while datatest['sqfeet'].max() > 10*(datatest['sqfeet'].mean() + datatest['sqfeet'].std()): #Aplicar esta modificacion hasta que el maximo no sea 10 veces más que la suma de la media y la desviación estandar 
    datatest = datatest.replace(datatest['sqfeet'].max() , int(datatest['sqfeet'].mean())) #Cambiamos el valor del outlier a la media para que no se desvíe la curva y sea más óptima la predicción


In [32]:
XTest = datatest[['sqfeet','baths','beds','comes_furnished']]

In [33]:
#Predecimos usando los valores nuevos de los atributos y el objeto anteriormente creado 'Arbol'
Pred = Arbol.predict(XTest) 

/home/justo99/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [34]:
pred = pd.DataFrame(Pred)

In [35]:
pred.to_csv('JustoCosta.csv' , sep = ',', index = False)